In [1]:
import yaml
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, col, max, min, count, broadcast, desc, asc, when, lit,row_number, rank, round
from pyspark.sql.window import Window
from pyspark.sql.types import StringType, IntegerType, FloatType

In [2]:
def control_null (dataframe): 
    for dfcol in dataframe.columns:
        df_null = dataframe.filter(col(dfcol).isNull()).count()
        if df_null > 0:
            print(f"{dfcol} : {df_null} null")
        else:
            print(f"{dfcol} no tiene null")

In [3]:
spark = (
    SparkSession.builder
    .appName("Spark")
    .config("spark.driver.extraJavaOptions", r'-Dlog4j.configurationFile=file:/home/illidan/proyecto_desde0/ETL/log4j.properties')\
    .getOrCreate()
)

spark.sparkContext.setLogLevel("INFO")

logger = spark._jvm.org.apache.log4j.LogManager.getLogger(__name__)

logger.info("Log de ejemplo guardado en archivo y consola.")

errores_detectados = []


your 131072x1 screen size is bogus. expect trouble
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


13:47:08.628 [Thread-3] INFO  __main__ - Log de ejemplo guardado en archivo y consola.


In [4]:
try:
    #reading the config file
    #geting file path into a dictionary
    with open("/home/illidan/proyecto_desde0/Config_file/Config.Yaml", "r") as file:
        config = yaml.safe_load(file)
except Exception as error:
    logger.error("Error ruta .yaml:" + str(error))
    
try:
    read_parquet_airline = config["Parquet_file"]["df_airline"]
    read_parquet_flights = config["Parquet_file"]["df_flights"]
    read_parquet_airports = config["Parquet_file"]["df_airports"]
except Exception as error:
    logger.error("Error ruta .yaml:" + str(error))


In [5]:
try:
    df_flights = spark.read.parquet(read_parquet_flights)
    df_airline = spark.read.parquet(read_parquet_airline)
    df_airports = spark.read.parquet(read_parquet_airports)
except Exception as error:
    logger.error("Error read.parquet:" + str(error))




flight_row_count = df_flights.count()

airline_row_count = df_airline.count()

airports_row_count = df_airports.count()

13:47:08.753 [Thread-3] INFO  org.apache.spark.sql.internal.SharedState - Setting hive.metastore.warehouse.dir ('null') to the value of spark.sql.warehouse.dir.
13:47:08.763 [Thread-3] INFO  org.apache.spark.sql.internal.SharedState - Warehouse path is 'file:/home/illidan/proyecto_desde0/ETL/spark-warehouse'.
13:47:08.778 [Thread-3] INFO  org.sparkproject.jetty.server.handler.ContextHandler - Started o.s.j.s.ServletContextHandler@4a338390{/SQL,null,AVAILABLE,@Spark}
13:47:08.780 [Thread-3] INFO  org.sparkproject.jetty.server.handler.ContextHandler - Started o.s.j.s.ServletContextHandler@1b04bc19{/SQL/json,null,AVAILABLE,@Spark}
13:47:08.781 [Thread-3] INFO  org.sparkproject.jetty.server.handler.ContextHandler - Started o.s.j.s.ServletContextHandler@6507af6f{/SQL/execution,null,AVAILABLE,@Spark}
13:47:08.782 [Thread-3] INFO  org.sparkproject.jetty.server.handler.ContextHandler - Started o.s.j.s.ServletContextHandler@43fd4f69{/SQL/execution/json,null,AVAILABLE,@Spark}
13:47:08.794 [Threa

13:47:11.148 [Executor task launch worker for task 0.0 in stage 0.0 (TID 0)] INFO  org.apache.spark.executor.Executor - Finished task 0.0 in stage 0.0 (TID 0). 3053 bytes result sent to driver
13:47:11.160 [task-result-getter-0] INFO  org.apache.spark.scheduler.TaskSetManager - Finished task 0.0 in stage 0.0 (TID 0) in 491 ms on 172.23.57.81 (executor driver) (1/1)
13:47:11.162 [task-result-getter-0] INFO  org.apache.spark.scheduler.TaskSchedulerImpl - Removed TaskSet 0.0, whose tasks have all completed, from pool 
13:47:11.167 [dag-scheduler-event-loop] INFO  org.apache.spark.scheduler.DAGScheduler - ResultStage 0 (parquet at NativeMethodAccessorImpl.java:0) finished in 0.704 s
13:47:11.171 [dag-scheduler-event-loop] INFO  org.apache.spark.scheduler.DAGScheduler - Job 0 is finished. Cancelling potential speculative or zombie tasks for this job
13:47:11.171 [dag-scheduler-event-loop] INFO  org.apache.spark.scheduler.TaskSchedulerImpl - Killing all running tasks in stage 0: Stage finish

13:47:14.331 [Executor task launch worker for task 0.0 in stage 5.0 (TID 11)] INFO  org.apache.spark.storage.ShuffleBlockFetcherIterator - Getting 8 (480.0 B) non-empty blocks including 8 (480.0 B) local and 0 (0.0 B) host-local and 0 (0.0 B) push-merged-local and 0 (0.0 B) remote blocks
13:47:14.334 [Executor task launch worker for task 0.0 in stage 5.0 (TID 11)] INFO  org.apache.spark.storage.ShuffleBlockFetcherIterator - Started 0 remote fetches in 14 ms
13:47:14.355 [Executor task launch worker for task 0.0 in stage 5.0 (TID 11)] INFO  org.apache.spark.sql.catalyst.expressions.codegen.CodeGenerator - Code generated in 15.526386 ms
13:47:14.386 [Executor task launch worker for task 0.0 in stage 5.0 (TID 11)] INFO  org.apache.spark.executor.Executor - Finished task 0.0 in stage 5.0 (TID 11). 4081 bytes result sent to driver
13:47:14.389 [task-result-getter-1] INFO  org.apache.spark.scheduler.TaskSetManager - Finished task 0.0 in stage 5.0 (TID 11) in 132 ms on 172.23.57.81 (executor 

13:47:14.540 [Executor task launch worker for task 0.0 in stage 6.0 (TID 12)] INFO  org.apache.spark.executor.Executor - Finished task 0.0 in stage 6.0 (TID 12). 2136 bytes result sent to driver
13:47:14.544 [task-result-getter-2] INFO  org.apache.spark.scheduler.TaskSetManager - Finished task 0.0 in stage 6.0 (TID 12) in 26 ms on 172.23.57.81 (executor driver) (1/1)
13:47:14.545 [task-result-getter-2] INFO  org.apache.spark.scheduler.TaskSchedulerImpl - Removed TaskSet 6.0, whose tasks have all completed, from pool 
13:47:14.546 [dag-scheduler-event-loop] INFO  org.apache.spark.scheduler.DAGScheduler - ShuffleMapStage 6 (count at NativeMethodAccessorImpl.java:0) finished in 0.048 s
13:47:14.560 [dag-scheduler-event-loop] INFO  org.apache.spark.scheduler.DAGScheduler - looking for newly runnable stages
13:47:14.560 [dag-scheduler-event-loop] INFO  org.apache.spark.scheduler.DAGScheduler - running: Set()
13:47:14.560 [dag-scheduler-event-loop] INFO  org.apache.spark.scheduler.DAGSchedul

In [6]:
try:
    #avg per fly
    avg_flight = df_flights.join(broadcast(df_airline), df_flights.AIRLINE == df_airline.IATA_CODE) \
                        .groupBy(df_flights.AIRLINE, df_airline.AIRLINE) \
                        .agg(round(avg(df_flights.DISTANCE), 2).alias("avg_DISTANCE")) \
                        .orderBy(desc("avg_DISTANCE")) \
                        .select(df_airline.AIRLINE.alias("AIRLINE_Name"), "avg_DISTANCE")
except Exception as error:
    logger.error("Error variable avg_flight:" + str(error))


In [16]:
try:
    #how many times a flight visit an airport
    airline_in_airport = df_flights.join(broadcast(df_airports), df_flights.ORIGIN_AIRPORT == df_airports.IATA_CODE) \
                                .join(broadcast(df_airline), df_flights.AIRLINE == df_airline.IATA_CODE) \
                                .repartition(df_flights.DESTINATION_AIRPORT) \
                                .groupBy(df_flights.DESTINATION_AIRPORT, df_airline.AIRLINE) \
                                    .agg(count(df_flights.AIRLINE).alias("Count_visit_per_airline")) \
                                .orderBy(asc(df_flights.DESTINATION_AIRPORT)) \
                                .select(df_flights.DESTINATION_AIRPORT, df_airline.AIRLINE,"Count_visit_per_airline")
except Exception as error:
    logger.error("Error variable airline_in_airport:" + str(error))

#a Rank of wich airline is the most visited in each airport
try:
    window_spec = Window.partitionBy("DESTINATION_AIRPORT").orderBy(desc("Count_visit_per_airline"))
    
    Rank_airline_in_airport = airline_in_airport.withColumn("Ranking", rank().over(window_spec))


except Exception as error:
    logger.error("Error Rank_airline_in_airport:" + str(error))


In [8]:
try:
    #mention how many times an airline visit a destination and canceled this arrival
    flights_per_cancell = df_flights.select(col("AIRLINE"), col("CANCELLED"), col("DESTINATION_AIRPORT")) \
                                .filter(col("CANCELLED") == "1") \
                                .repartition(col("AIRLINE"), col("DESTINATION_AIRPORT")) \
                                .groupBy(col("AIRLINE"), col("DESTINATION_AIRPORT")) \
                                    .agg(count(col("AIRLINE")).alias("count_airlines_cancel")) \
                                .orderBy(desc("DESTINATION_AIRPORT")) \
                                .limit(100)
except Exception as error:
    logger.error("Error variable flights_per_cancell:" + str(error))


In [9]:
try:
    #i create this variable because i need to know how many airports have flights but are not in the list of airports
    airport_notin_thelist = df_flights.join(broadcast(df_airports), df_flights.DESTINATION_AIRPORT == df_airports.IATA_CODE, "left_anti") \
                                        .select("DESTINATION_AIRPORT") \
                                        .repartition("DESTINATION_AIRPORT") \
                                        .groupBy("DESTINATION_AIRPORT").agg(count("*").alias("Count_airports"))
except Exception as error:
    logger.error("Error variable airport_notin_thelist:" + str(error))


In [11]:
#Create parquet files

try:
    avg_flight.write.parquet(config["Parquet_file"]["avg_flight"], mode="overwrite")

    airline_in_airport.write.parquet(config["Parquet_file"]["airline_in_airport"], mode="overwrite")

    flights_per_cancell.write.parquet(config["Parquet_file"]["flights_per_cancell"], mode="overwrite")

    airport_notin_thelist.write.parquet(config["Parquet_file"]["airport_notin_thelist"], mode="overwrite")
except Exception as error:
    logger.error("Error write parquet files:" + str(error))


13:47:15.897 [Thread-3] INFO  org.apache.spark.sql.execution.datasources.FileSourceStrategy - Pushed Filters: IsNotNull(AIRLINE)
13:47:15.897 [Thread-3] INFO  org.apache.spark.sql.execution.datasources.FileSourceStrategy - Post-Scan Filters: isnotnull(AIRLINE#4)
13:47:15.902 [Thread-3] INFO  org.apache.spark.sql.execution.datasources.FileSourceStrategy - Pushed Filters: IsNotNull(IATA_CODE)
13:47:15.902 [Thread-3] INFO  org.apache.spark.sql.execution.datasources.FileSourceStrategy - Post-Scan Filters: isnotnull(IATA_CODE#62)
13:47:15.954 [dispatcher-BlockManagerMaster] INFO  org.apache.spark.storage.BlockManagerInfo - Removed broadcast_10_piece0 on 172.23.57.81:32937 in memory (size: 7.8 KiB, free: 434.3 MiB)
13:47:15.959 [dispatcher-BlockManagerMaster] INFO  org.apache.spark.storage.BlockManagerInfo - Removed broadcast_11_piece0 on 172.23.57.81:32937 in memory (size: 5.9 KiB, free: 434.3 MiB)
13:47:16.026 [broadcast-exchange-0] INFO  org.apache.spark.sql.catalyst.expressions.codegen.C

13:47:18.577 [Executor task launch worker for task 6.0 in stage 13.0 (TID 23)] INFO  org.apache.spark.executor.Executor - Finished task 6.0 in stage 13.0 (TID 23). 3926 bytes result sent to driver
13:47:18.582 [task-result-getter-0] INFO  org.apache.spark.scheduler.TaskSetManager - Finished task 6.0 in stage 13.0 (TID 23) in 1810 ms on 172.23.57.81 (executor driver) (1/8)
13:47:18.583 [Executor task launch worker for task 0.0 in stage 13.0 (TID 17)] INFO  org.apache.spark.executor.Executor - Finished task 0.0 in stage 13.0 (TID 17). 3926 bytes result sent to driver
13:47:18.584 [Executor task launch worker for task 7.0 in stage 13.0 (TID 24)] INFO  org.apache.spark.executor.Executor - Finished task 7.0 in stage 13.0 (TID 24). 3926 bytes result sent to driver
13:47:18.587 [task-result-getter-3] INFO  org.apache.spark.scheduler.TaskSetManager - Finished task 0.0 in stage 13.0 (TID 17) in 1818 ms on 172.23.57.81 (executor driver) (2/8)
13:47:18.589 [task-result-getter-1] INFO  org.apache.

13:47:19.030 [Thread-3] INFO  org.apache.spark.sql.execution.datasources.parquet.ParquetUtils - Using default output committer for Parquet: org.apache.parquet.hadoop.ParquetOutputCommitter
13:47:19.043 [Thread-3] INFO  org.apache.hadoop.mapreduce.lib.output.FileOutputCommitter - File Output Committer Algorithm version is 1
13:47:19.044 [Thread-3] INFO  org.apache.hadoop.mapreduce.lib.output.FileOutputCommitter - FileOutputCommitter skip cleanup _temporary folders under output directory:false, ignore cleanup failures: false
13:47:19.044 [Thread-3] INFO  org.apache.spark.sql.execution.datasources.SQLHadoopMapReduceCommitProtocol - Using user defined output committer class org.apache.parquet.hadoop.ParquetOutputCommitter
13:47:19.045 [Thread-3] INFO  org.apache.hadoop.mapreduce.lib.output.FileOutputCommitter - File Output Committer Algorithm version is 1
13:47:19.045 [Thread-3] INFO  org.apache.hadoop.mapreduce.lib.output.FileOutputCommitter - FileOutputCommitter skip cleanup _temporary f

13:47:20.967 [Executor task launch worker for task 6.0 in stage 23.0 (TID 36)] INFO  org.apache.spark.executor.Executor - Finished task 6.0 in stage 23.0 (TID 36). 2494 bytes result sent to driver
13:47:20.980 [task-result-getter-3] INFO  org.apache.spark.scheduler.TaskSetManager - Finished task 6.0 in stage 23.0 (TID 36) in 1065 ms on 172.23.57.81 (executor driver) (1/8)


13:47:21.579 [Executor task launch worker for task 7.0 in stage 23.0 (TID 37)] INFO  org.apache.spark.executor.Executor - Finished task 7.0 in stage 23.0 (TID 37). 2494 bytes result sent to driver
13:47:21.590 [task-result-getter-1] INFO  org.apache.spark.scheduler.TaskSetManager - Finished task 7.0 in stage 23.0 (TID 37) in 1674 ms on 172.23.57.81 (executor driver) (2/8)
13:47:21.633 [Executor task launch worker for task 5.0 in stage 23.0 (TID 35)] INFO  org.apache.spark.executor.Executor - Finished task 5.0 in stage 23.0 (TID 35). 2451 bytes result sent to driver
13:47:21.635 [task-result-getter-2] INFO  org.apache.spark.scheduler.TaskSetManager - Finished task 5.0 in stage 23.0 (TID 35) in 1720 ms on 172.23.57.81 (executor driver) (3/8)
13:47:21.717 [Executor task launch worker for task 1.0 in stage 23.0 (TID 31)] INFO  org.apache.spark.executor.Executor - Finished task 1.0 in stage 23.0 (TID 31). 2494 bytes result sent to driver
13:47:21.718 [task-result-getter-0] INFO  org.apache.

...

13:47:23.109 [Thread-3] INFO  org.apache.spark.sql.catalyst.expressions.codegen.CodeGenerator - Code generated in 12.62907 ms
13:47:23.114 [Thread-3] INFO  org.apache.spark.storage.memory.MemoryStore - Block broadcast_30 stored as values in memory (estimated size 200.5 KiB, free 429.1 MiB)
13:47:23.124 [Thread-3] INFO  org.apache.spark.storage.memory.MemoryStore - Block broadcast_30_piece0 stored as bytes in memory (estimated size 34.8 KiB, free 429.0 MiB)
13:47:23.125 [dispatcher-BlockManagerMaster] INFO  org.apache.spark.storage.BlockManagerInfo - Added broadcast_30_piece0 in memory on 172.23.57.81:32937 (size: 34.8 KiB, free: 434.1 MiB)
13:47:23.126 [Thread-3] INFO  org.apache.spark.SparkContext - Created broadcast 30 from parquet at NativeMethodAccessorImpl.java:0
13:47:23.127 [Thread-3] INFO  org.apache.spark.sql.execution.FileSourceScanExec - Planning scan with bin packing, max size: 21880221 bytes, open cost is considered as scanning 4194304 bytes.
13:47:23.131 [dag-scheduler-ev

13:47:23.834 [Executor task launch worker for task 6.0 in stage 27.0 (TID 55)] INFO  org.apache.spark.executor.Executor - Finished task 6.0 in stage 27.0 (TID 55). 2296 bytes result sent to driver
13:47:23.839 [task-result-getter-3] INFO  org.apache.spark.scheduler.TaskSetManager - Finished task 6.0 in stage 27.0 (TID 55) in 693 ms on 172.23.57.81 (executor driver) (2/8)
13:47:24.009 [Executor task launch worker for task 1.0 in stage 27.0 (TID 50)] INFO  org.apache.spark.executor.Executor - Finished task 1.0 in stage 27.0 (TID 50). 2296 bytes result sent to driver
13:47:24.010 [task-result-getter-1] INFO  org.apache.spark.scheduler.TaskSetManager - Finished task 1.0 in stage 27.0 (TID 50) in 865 ms on 172.23.57.81 (executor driver) (3/8)


13:47:24.108 [Executor task launch worker for task 2.0 in stage 27.0 (TID 51)] INFO  org.apache.spark.executor.Executor - Finished task 2.0 in stage 27.0 (TID 51). 2296 bytes result sent to driver
13:47:24.110 [task-result-getter-2] INFO  org.apache.spark.scheduler.TaskSetManager - Finished task 2.0 in stage 27.0 (TID 51) in 965 ms on 172.23.57.81 (executor driver) (4/8)
13:47:24.153 [Executor task launch worker for task 3.0 in stage 27.0 (TID 52)] INFO  org.apache.spark.executor.Executor - Finished task 3.0 in stage 27.0 (TID 52). 2296 bytes result sent to driver
13:47:24.154 [task-result-getter-0] INFO  org.apache.spark.scheduler.TaskSetManager - Finished task 3.0 in stage 27.0 (TID 52) in 1008 ms on 172.23.57.81 (executor driver) (5/8)
13:47:24.155 [Executor task launch worker for task 0.0 in stage 27.0 (TID 49)] INFO  org.apache.spark.executor.Executor - Finished task 0.0 in stage 27.0 (TID 49). 2296 bytes result sent to driver
13:47:24.155 [Executor task launch worker for task 5.0

13:47:24.735 [dag-scheduler-event-loop] INFO  org.apache.spark.scheduler.DAGScheduler - Registering RDD 78 (parquet at NativeMethodAccessorImpl.java:0) as input to shuffle 8
13:47:24.735 [dag-scheduler-event-loop] INFO  org.apache.spark.scheduler.DAGScheduler - Got map stage job 21 (parquet at NativeMethodAccessorImpl.java:0) with 8 output partitions
13:47:24.735 [dag-scheduler-event-loop] INFO  org.apache.spark.scheduler.DAGScheduler - Final stage: ShuffleMapStage 31 (parquet at NativeMethodAccessorImpl.java:0)
13:47:24.735 [dag-scheduler-event-loop] INFO  org.apache.spark.scheduler.DAGScheduler - Parents of final stage: List()
13:47:24.735 [dag-scheduler-event-loop] INFO  org.apache.spark.scheduler.DAGScheduler - Missing parents: List()
13:47:24.736 [dag-scheduler-event-loop] INFO  org.apache.spark.scheduler.DAGScheduler - Submitting ShuffleMapStage 31 (MapPartitionsRDD[78] at parquet at NativeMethodAccessorImpl.java:0), which has no missing parents
13:47:24.740 [dag-scheduler-event-

13:47:25.422 [Executor task launch worker for task 5.0 in stage 31.0 (TID 64)] INFO  org.apache.spark.executor.Executor - Finished task 5.0 in stage 31.0 (TID 64). 2339 bytes result sent to driver
13:47:25.423 [task-result-getter-0] INFO  org.apache.spark.scheduler.TaskSetManager - Finished task 5.0 in stage 31.0 (TID 64) in 676 ms on 172.23.57.81 (executor driver) (7/8)
13:47:25.484 [Executor task launch worker for task 6.0 in stage 31.0 (TID 65)] INFO  org.apache.spark.executor.Executor - Finished task 6.0 in stage 31.0 (TID 65). 2339 bytes result sent to driver
13:47:25.485 [task-result-getter-1] INFO  org.apache.spark.scheduler.TaskSetManager - Finished task 6.0 in stage 31.0 (TID 65) in 737 ms on 172.23.57.81 (executor driver) (8/8)
13:47:25.485 [task-result-getter-1] INFO  org.apache.spark.scheduler.TaskSchedulerImpl - Removed TaskSet 31.0, whose tasks have all completed, from pool 
13:47:25.485 [dag-scheduler-event-loop] INFO  org.apache.spark.scheduler.DAGScheduler - ShuffleMap